In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from rpad.rlbench_utils.placement_dataset import RLBenchPlacementDataset, load_handle_mapping, load_state_pos_dict
import numpy as np

In [ ]:
dset = RLBenchPlacementDataset(
    dataset_root="/home/beisner/datasets/rlbench/",
    # task_name="stack_wine",
    task_name="insert_onto_square_peg",
    # task_name="insert_usb_in_computer",
    # task_name="phone_on_base",
    # task_name="put_toilet_roll_on_stand",
    n_demos=1,
    phase="place",
)

In [ ]:
dset.names_to_handles

In [ ]:
dset[0]

In [ ]:
np.unique(dset.demos[0][0].right_shoulder_mask)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%matplotlib widget

In [ ]:
# plt.imshow(dset.demos[0][-1].wrist_mask == 116)

# Show a grid of images from left, right, front, overhead, and wrist cameras
fig, axs = plt.subplots(2, 5, figsize=(10, 4))
(ax1, ax2, ax3, ax4, ax5), (ax6, ax7, ax8, ax9, ax10) = axs

ax1.imshow(dset.demos[0][-1].left_shoulder_mask)
ax1.set_title("Left Shoulder Camera")

ax2.imshow(dset.demos[0][-1].right_shoulder_mask)
ax2.set_title("Right Shoulder Camera")

ax3.imshow(dset.demos[0][-1].front_mask)
ax3.set_title("Front Camera")

ax4.imshow(dset.demos[0][-1].overhead_mask)
ax4.set_title("Overhead Camera")

ax5.imshow(dset.demos[0][-1].wrist_mask)
ax5.set_title("Wrist Camera")

# Now plot only those which match the object mask
id = 98
ax6.imshow(dset.demos[0][-1].left_shoulder_mask == id)
ax6.set_title("Left Shoulder Camera")

ax7.imshow(dset.demos[0][-1].right_shoulder_mask == id)
ax7.set_title("Right Shoulder Camera")

ax8.imshow(dset.demos[0][-1].front_mask == id)
ax8.set_title("Front Camera")

ax9.imshow(dset.demos[0][-1].overhead_mask == id)
ax9.set_title("Overhead Camera")

ax10.imshow(dset.demos[0][-1].wrist_mask == id)
ax10.set_title("Wrist Camera")

plt.tight_layout()



plt.show()

In [ ]:
import open3d as o3d
from open3d.web_visualizer import draw

cube_red = o3d.geometry.TriangleMesh.create_box(1, 2, 4)
cube_red.compute_vertex_normals()
cube_red.paint_uniform_color((1.0, 0.0, 0.0))
draw(cube_red)

In [ ]:
import open3d as o3d
import open3d.web_visualizer as wo3d

data = dset[0]

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(data["key_action_pc"])
# make it red
pcd.colors = o3d.utility.Vector3dVector(np.tile(np.array([1, 0, 0]), (data["key_action_pc"].shape[0], 1)))

pcd2 = o3d.geometry.PointCloud()
pcd2.points = o3d.utility.Vector3dVector(data["key_anchor_pc"])
# make it blue
pcd2.colors = o3d.utility.Vector3dVector(np.tile(np.array([0, 0, 1]), (data["key_anchor_pc"].shape[0], 1)))

pcd3 = o3d.geometry.PointCloud()
pcd3.points = o3d.utility.Vector3dVector(data["init_action_pc"])
# make it green
pcd3.colors = o3d.utility.Vector3dVector(np.tile(np.array([0, 1, 0]), (data["init_action_pc"].shape[0], 1)))

init_action_pc = data["init_action_pc"]
T_init_key = data["T_init_key"]

# Transform the init_action_pc
init_action_pc = np.hstack((init_action_pc, np.ones((init_action_pc.shape[0], 1))))
tfm_init_action_pc = np.matmul(T_init_key, init_action_pc.T).T[:, :3]

pcd4 = o3d.geometry.PointCloud()
pcd4.points = o3d.utility.Vector3dVector(tfm_init_action_pc)
# make it green
pcd4.colors = o3d.utility.Vector3dVector(np.tile(np.array([0, 1, 0]), (tfm_init_action_pc.shape[0], 1)))

# o3d.visualization.draw_geometries([
wo3d.draw([
    pcd, 
    pcd2,
    pcd3,
    pcd4,
])

In [ ]:
data = dset[0]
data["key_action_pc"].shape

In [ ]:
data["anchor_pc"].shape

In [ ]:
dset.demos[0][-1].misc

In [ ]:
state_pos_dict = load_state_pos_dict(dset.dataset_root, "insert_onto_square_peg", 0, 0)
state_pos_dict

In [ ]:
load_handle_mapping(dset.dataset_root, "put_toilet_roll_on_stand", 0)

In [ ]:
for k in dset.demos[0][-1].misc.keys():
    print(k)

In [ ]:
dir(dset.demos[0][-1])

In [ ]:
dset.demos[0][-1].get_low_dim_data()

In [ ]:
print(dset.demos[0][0].task_low_dim_state)

In [ ]:
dset.demos[0][0].task_low_dim_state

In [ ]:
start = state_pos_dict["wine_bottle"]
end = start + 7
p = dset.demos[0][-1].task_low_dim_state[start:end]

In [ ]:
t, q = p[:3], p[3:]

In [ ]:
t

In [ ]:
dset.demos[0][0].gripper_pose.shape